# Golly Sportsbook

## SQL Game Outcome ETL


In [1]:
import os, sys, subprocess, json, time
from sqlalchemy import Column, Integer, String, Boolean, Table
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm import DeclarativeBase

In [2]:
class Base(DeclarativeBase):
    pass

In [3]:
class GameResult(Base):
    __tablename__ = "sportsbook_game_result"
    gameid = Column("gameid", String, primary_key=True)
    cup = Column("cup", String)
    season = Column("season", Integer)
    day = Column("day", Integer)
    mapName = Column("mapName", String)
    winnerAbbr = Column("winnerAbbr", String)
    winnerPoints = Column("winnerPoints", Integer)
    loserAbbr = Column("loserAbbr", String)
    loserPoints = Column("loserPoints", Integer)
    isPostseason = Column("isPostseason", Boolean)
    series = Column("series", String)
    generations = Column("generations", Integer)

In [4]:
def fetch_data(which_season0, cup, fname):
    f = os.path.join('..', 'data', f'gollyx-{cup}-data', f'season{which_season0}', fname)
    if not os.path.exists(f):
        raise Exception(f"Error: season {which_season0} not valid: {f} does not exist")
    with open(f, 'r') as fz:
        season0_seas = json.load(fz)
    return season0_seas

def fetch_season_data(which_season0, cup):
    return fetch_data(which_season0, cup, fname='season.json')

def fetch_postseason_data(which_season0, cup):
    return fetch_data(which_season0, cup, fname='postseason.json')

In [5]:
def flatten_season(season):
    games = []
    for day in season:
        for game in day:
            games.append(game)
    return games

def flatten_postseason(postseason):
    games = []
    for series in postseason:
        miniseason = postseason[series]
        for day in miniseason:
            for game in day:
                game['series'] = series
                games.append(game)
    return games

In [6]:
def create_new_game_outcome(session, game, cup, id_key='gameid'):
    try:
        this_id = game[id_key]
    except KeyError:
        id_key = 'id'
        this_id = game[id_key]
    
    # Check if game already exists
    existing_game = (
        session.query(GameResult)
        .filter(GameResult.gameid==this_id)
        .one_or_none()
    )
    if existing_game is not None:
        # print(f"Found an existing game with game id {this_id}, skipping insert")
        return

    # Now create the new game.
    if 'series' not in game:
        game['series'] = ''
    if 'cup' not in game:
        game['cup'] = cup

    # Keys are mostly one-to-one map, except maps and win/loss.
    # (First, account for the difference in keys in postseason vs regular season games)
    if 'description' not in game:
        game['description'] = ''

    if game['team1Score'] > game['team2Score']:
        
        game['winnerAbbr'] = game['team1Abbr']
        game['winnerPoints'] = game['team1Score']

        game['loserAbbr'] = game['team2Abbr']
        game['loserPoints'] = game['team2Score']
        
    elif game['team2Score'] > game['team1Score']:
        
        game['winnerAbbr'] = game['team2Abbr']
        game['winnerPoints'] = game['team2Score']

        game['loserAbbr'] = game['team1Abbr']
        game['loserPoints'] = game['team1Score']
    
    g = GameResult(
        gameid = game[id_key],
        cup = game['cup'],
        season = game['season'],
        day = game['day'],
        mapName = game['map']['mapName'],
        winnerAbbr = game['winnerAbbr'],
        winnerPoints = game['winnerPoints'],
        loserAbbr = game['loserAbbr'],
        loserPoints = game['loserPoints'],
        isPostseason = game['isPostseason'],
        series = game['series'],
        generations = game['generations']
    )
    session.add(g)

In [7]:
import logging

logging.basicConfig()
logging.getLogger("sqlalchemy.engine").setLevel(logging.WARNING)
#logging.getLogger("sqlalchemy.pool").setLevel(logging.DEBUG)
logging.getLogger("sqlalchemy.pool").setLevel(logging.WARNING)

In [8]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

fpath = os.path.join(os.environ['HOME'], 'tmp', 'sqlite', 'mydb.db')
engine = create_engine(f"sqlite:///{fpath}")
Session = sessionmaker(engine)

In [9]:
# Start by creating all the tables
Base.metadata.create_all(engine)

In [10]:
# Run a simple query on the tables
with Session() as session:
    results = session.query(GameResult).all()

print(len(results))

0


In [11]:
CUPS = ['hellmouth', 'pseudo', 'toroidal', 'star', 'klein']

In [12]:
for CUP in CUPS:
    print(f"\n{CUP}:\n")
    
    for season0 in range(24):
        sdat = flatten_season(fetch_season_data(season0, CUP))
        # Create a new context manager for each season, to limit number of inserts at one time
        with Session.begin() as session:
            print(f"Working on season0={season0}")
            for game in sdat:
                create_new_game_outcome(session, game, CUP)
    print("\nDone!\n")
    
    for season0 in range(24):
        pdat = flatten_postseason(fetch_postseason_data(season0, CUP))
        # Create a new context manager for each season, to limit number of inserts at one time
        with Session.begin() as session:
            print(f"Working on season0={season0} postseason")
            for game in pdat:
                create_new_game_outcome(session, game, CUP)
    print("\nDone!\n")


hellmouth:

Working on season0=0
Working on season0=1
Working on season0=2
Working on season0=3
Working on season0=4
Working on season0=5
Working on season0=6
Working on season0=7
Working on season0=8
Working on season0=9
Working on season0=10
Working on season0=11
Working on season0=12
Working on season0=13
Working on season0=14
Working on season0=15
Working on season0=16
Working on season0=17
Working on season0=18
Working on season0=19
Working on season0=20
Working on season0=21
Working on season0=22
Working on season0=23

Done!

Working on season0=0 postseason
Working on season0=1 postseason
Working on season0=2 postseason
Working on season0=3 postseason
Working on season0=4 postseason
Working on season0=5 postseason
Working on season0=6 postseason
Working on season0=7 postseason
Working on season0=8 postseason
Working on season0=9 postseason
Working on season0=10 postseason
Working on season0=11 postseason
Working on season0=12 postseason
Working on season0=13 postseason
Working o